In [1]:
!pip install geopandas folium -q

In [11]:
!pip install osmnx -q
import geopandas as gpd
import pandas as pd
import folium
from shapely.geometry import Point
from IPython.display import display
import osmnx as ox

data = [
    ["สถานีขนส่งผู้โดยสารจังหวัดอุบลราชธานี",15.2727387,104.8372651],
    ["สถานีขนส่งผู้โดยสารอำเภอเดชอุดม",14.898639,105.069111],
    ["สถานีขนส่งผู้โดยสารอำเภอเขมราฐ",16.0376306,105.2220817],
    ["สถานีขนส่งผู้โดยสารระหว่างประเทศ ช่องเม็ก",15.133361,105.461583],
    ["ขนส่งอำเภอเขื่องใน",15.392167,104.553444],
    ["ท่ารถทัวร์ วัฒนานคร อ.โขงเจียม",15.318444,105.491306]
]

df = pd.DataFrame(data,columns=["name","lat","lon"])
df["source"] = "Google Maps"
geometry = [Point(xy) for xy in zip(df.lon,df.lat)]
stations = gpd.GeoDataFrame(df,geometry=geometry,crs="EPSG:4326")

stations

,name,lat,lon,source,geometry
0,สถานีขนส่งผู้โดยสารจังหวัดอุบลราชธานี,15.272739,104.837265,Google Maps,POINT (104.83727 15.27274)
1,สถานีขนส่งผู้โดยสารอำเภอเดชอุดม,14.898639,105.069111,Google Maps,POINT (105.06911 14.89864)
2,สถานีขนส่งผู้โดยสารอำเภอเขมราฐ,16.037631,105.222082,Google Maps,POINT (105.22208 16.03763)
3,สถานีขนส่งผู้โดยสารระหว่างประเทศ ช่องเม็ก,15.133361,105.461583,Google Maps,POINT (105.46158 15.13336)
4,ขนส่งอำเภอเขื่องใน,15.392167,104.553444,Google Maps,POINT (104.55344 15.39217)
5,ท่ารถทัวร์ วัฒนานคร อ.โขงเจียม,15.318444,105.491306,Google Maps,POINT (105.49131 15.31844)


In [13]:
city = ox.geocode_to_gdf("Ubon Ratchathani Province, Thailand")
city = city.to_crs(epsg=4326)
city

,geometry,bbox_west,bbox_south,bbox_east,bbox_north,place_id,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name
0,"POLYGON ((104.37158 15.45805, 104.3723 15.4574...",104.371584,14.209568,105.636812,16.098149,238150946,relation,1908830,15.187968,105.326954,boundary,administrative,8,0.512925,province,Ubon Ratchathani Province,"Ubon Ratchathani Province, Thailand"


In [8]:
stations_utm = stations.to_crs(epsg=32648)

stations_utm["buffer_geometry"] = stations_utm.buffer(5000)


buffer_gdf = gpd.GeoDataFrame(
    stations_utm[['name']],
    geometry=stations_utm['buffer_geometry'],
    crs=stations_utm.crs
)

buffer_wgs = buffer_gdf.to_crs(epsg=4326)

In [31]:
city_utm = city.to_crs(epsg=32648)
impact = gpd.overlay(city_utm, buffer_gdf, how="intersection")
impact["area_km2"] = impact.area / 1e6
impact[["name_2","area_km2"]]

,name_2,area_km2
0,สถานีขนส่งผู้โดยสารจังหวัดอุบลราชธานี,78.413712
1,สถานีขนส่งผู้โดยสารอำเภอเดชอุดม,78.413712
2,สถานีขนส่งผู้โดยสารอำเภอเขมราฐ,51.532787
3,สถานีขนส่งผู้โดยสารระหว่างประเทศ ช่องเม็ก,54.931172
4,ขนส่งอำเภอเขื่องใน,78.413712
5,ท่ารถทัวร์ วัฒนานคร อ.โขงเจียม,52.598494


In [24]:
center_lat = stations.lat.mean()
center_lon = stations.lon.mean()

m = folium.Map(location=[center_lat,center_lon],zoom_start=9)

folium.GeoJson(
    city.__geo_interface__,
    style_function=lambda x:{
        "color":"blue",
        "weight":2,
        "fillOpacity":0.05
    }
).add_to(m)

folium.GeoJson(
    buffer_wgs.__geo_interface__,
    style_function=lambda x:{
        "color":"orange",
        "fillColor":"orange",
        "fillOpacity":0.25,
        "weight":3
    }
).add_to(m)

for idx,row in stations.iterrows():

    folium.CircleMarker(
        location=[row.lat,row.lon],
        radius=7,
        color="red",
        fill=True,
        fill_color="red",
        popup=row["name"],
        tooltip=folium.Tooltip(
            row["name"],
            style="""
                font-size:16px;
                font-weight:bold;
                color:black;
            """
        )
    ).add_to(m)

legend_html = """
<div style="
position: fixed;
bottom: 50px; left: 50px;
width: 250px;
background-color: white;
border:2px solid grey;
z-index:9999;
font-size:16px;
padding:10px;
">
<b>Map Legend</b><br>
🔴 Station<br>
🟠 Buffer (5 km)<br>
🔵 Ubon Ratchathani Province
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

display(m)

# **สรุปผลการเรียนรู้**

ในการทำ Lab นี้ได้เรียนรู้การใช้ GeoPandas เพื่อจัดการข้อมูลเชิงพื้นที่ โดยเริ่มจากการสร้างตำแหน่งสถานีขนส่งในจังหวัดอุบลราชธานีให้อยู่ในรูปแบบ GeoDataFrame และกำหนดระบบพิกัด EPSG:4326 จากนั้นได้ทำการแปลงระบบพิกัดเป็น UTM (EPSG:32648) เพื่อสร้าง Buffer ระยะ 5 กิโลเมตรรอบสถานีขนส่งแต่ละแห่ง ซึ่งช่วยให้สามารถวิเคราะห์พื้นที่ให้บริการได้อย่างถูกต้องตามหน่วยระยะทางจริง

หลังจากนั้นได้ใช้คำสั่ง overlay (Spatial Analysis) เพื่อหาพื้นที่ที่ Buffer ซ้อนทับกับขอบเขตจังหวัดอุบลราชธานี และคำนวณพื้นที่ผลกระทบ (Impact Area) ออกมาเป็นหน่วยตารางกิโลเมตร พบว่าค่าพื้นที่ให้บริการมีความแตกต่างกัน เช่น ประมาณ 78.41 km² สำหรับสถานีที่อยู่บริเวณตอนกลางจังหวัด และประมาณ 51–54 km² สำหรับสถานีที่อยู่ใกล้ขอบเขตจังหวัด เนื่องจากวง Buffer บางส่วนอยู่นอก boundary จังหวัด ทำให้พื้นที่ที่ถูกนำมาคำนวณลดลง

นอกจากนี้ยังได้เรียนรู้การสร้างแผนที่แบบ Interactive ด้วย Folium เพื่อแสดงขอบเขตจังหวัด ตำแหน่งสถานี และพื้นที่ Buffer พร้อม Legend และ Tooltip ช่วยให้สามารถตีความข้อมูลเชิงพื้นที่ได้ง่ายขึ้น จากการวิเคราะห์พบว่าการกระจายตัวของสถานีมีลักษณะกระจายหลายทิศทาง และพื้นที่ให้บริการส่วนใหญ่ไม่ทับซ้อนกันมาก แสดงให้เห็นถึงโครงสร้างเครือข่ายขนส่งที่กระจายตัวในระดับจังหวัดมากกว่าการกระจุกตัวอยู่ในพื้นที่เดียว

# **คำถามท้าย Lab**

**1. อธิบายความแตกต่างระหว่าง Spatial Join และ Attribute Join**

Spatial Join เป็นการเชื่อมข้อมูลโดยใช้ตำแหน่งทางภูมิศาสตร์ เช่น จุดอยู่ภายใน polygon หรือพื้นที่ซ้อนทับกัน ส่วน Attribute Join เป็นการเชื่อมข้อมูลจากค่าคอลัมน์ที่เหมือนกัน เช่น ID หรือชื่อ โดยไม่ได้พิจารณาตำแหน่งทางพื้นที่


**2. ทำไมเราต้องแปลงระบบพิกัดก่อนใช้ buffer() ใน GeoPandas?**

เนื่องจากระบบพิกัดแบบ EPSG:4326 ใช้หน่วยเป็นองศา หากนำไปสร้าง buffer จะได้ระยะไม่ถูกต้อง จึงต้องแปลงเป็นระบบพิกัดที่มีหน่วยเป็นเมตร เช่น UTM (EPSG:32648) เพื่อให้การสร้าง buffer มีขนาดจริงตามระยะทางที่กำหนด


**3. หากต้องการวิเคราะห์พื้นที่ให้บริการระยะ 10 กิโลเมตร แทน 5 กิโลเมตร ต้องแก้ไขส่วนใดของโค้ด?**

ให้แก้ค่าระยะในคำสั่ง buffer จาก stations_utm.buffer(5000) เป็น stations_utm.buffer(10000) เนื่องจากหน่วยของระยะทางเป็นเมตร

**4. วิธีใดที่เหมาะสมที่สุดในการแสดงผลข้อมูลเชิงพื้นที่ในรูปแบบ Interactive Map? เพราะเหตุใด?**

การใช้ Folium เหมาะสมสำหรับการสร้าง Interactive Map เพราะสามารถแสดงผลแบบโต้ตอบได้ เช่น การ zoom, popup และ tooltip ทำให้ผู้ใช้งานสามารถดูรายละเอียดตำแหน่งสถานีและพื้นที่ buffer ได้ง่ายและเข้าใจข้อมูลเชิงพื้นที่ได้ชัดเจนมากขึ้น

**5. หากพบว่า Buffer ที่สร้างมีขนาดไม่ถูกต้อง อาจเกิดจากสาเหตุใดได้บ้าง และสามารถแก้ไขได้อย่างไร?**

อาจเกิดจากการใช้ระบบพิกัดที่ไม่ถูกต้อง เช่น ยังเป็น EPSG:4326 หรือไม่ได้แปลงเป็นหน่วยเมตรก่อนสร้าง buffer รวมถึงอาจใส่ค่าระยะทางผิด วิธีแก้คือ ตรวจสอบค่า CRS ของข้อมูลและใช้คำสั่ง .to_crs() เพื่อแปลงระบบพิกัดก่อนสร้าง buffer